<a href="https://colab.research.google.com/github/Deepikadhinakaran/AI_price_optima/blob/main/price_optima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import numpy as np
import pandas as pd

In [6]:
df=pd.read_csv('dynamic_pricing.csv')

In [7]:
print("shape:",df.shape)

shape: (1000, 10)


In [8]:
df.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


In [9]:
mc=df.isnull().sum()
dc=df.duplicated().sum()
print("missing value: ",mc)
print("duplicate value: ",dc)

missing value:  Number_of_Riders           0
Number_of_Drivers          0
Location_Category          0
Customer_Loyalty_Status    0
Number_of_Past_Rides       0
Average_Ratings            0
Time_of_Booking            0
Vehicle_Type               0
Expected_Ride_Duration     0
Historical_Cost_of_Ride    0
dtype: int64
duplicate value:  0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Number_of_Riders         1000 non-null   int64  
 1   Number_of_Drivers        1000 non-null   int64  
 2   Location_Category        1000 non-null   object 
 3   Customer_Loyalty_Status  1000 non-null   object 
 4   Number_of_Past_Rides     1000 non-null   int64  
 5   Average_Ratings          1000 non-null   float64
 6   Time_of_Booking          1000 non-null   object 
 7   Vehicle_Type             1000 non-null   object 
 8   Expected_Ride_Duration   1000 non-null   int64  
 9   Historical_Cost_of_Ride  1000 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 78.3+ KB


In [11]:
print(df.columns)
df.describe()

Index(['Number_of_Riders', 'Number_of_Drivers', 'Location_Category',
       'Customer_Loyalty_Status', 'Number_of_Past_Rides', 'Average_Ratings',
       'Time_of_Booking', 'Vehicle_Type', 'Expected_Ride_Duration',
       'Historical_Cost_of_Ride'],
      dtype='object')


,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Expected_Ride_Duration,Historical_Cost_of_Ride
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,60.372000,27.076000,50.031000,4.257220,99.58800,372.502623
std,23.701506,19.068346,29.313774,0.435781,49.16545,187.158756
min,20.000000,5.000000,0.000000,3.500000,10.00000,25.993449
25%,40.000000,11.000000,25.000000,3.870000,59.75000,221.365202
50%,60.000000,22.000000,51.000000,4.270000,102.00000,362.019426
75%,81.000000,38.000000,75.000000,4.632500,143.00000,510.497504
max,100.000000,89.000000,100.000000,5.000000,180.00000,836.116419


In [12]:
num_cols=df.select_dtypes(include=np.number).columns.tolist()
cat_cols=df.select_dtypes(exclude=np.number).columns.tolist()
print("Numerical Columns:")
for num in num_cols:
    print(num)
print("\nCategorical Columns:")
for cat in cat_cols:
    print(cat)

Numerical Columns:
Number_of_Riders
Number_of_Drivers
Number_of_Past_Rides
Average_Ratings
Expected_Ride_Duration
Historical_Cost_of_Ride

Categorical Columns:
Location_Category
Customer_Loyalty_Status
Time_of_Booking
Vehicle_Type


In [13]:
# IQR Method
num_cols = df.select_dtypes(include=["int64","float64"]).columns

outlier_summary = {}

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)][col]
    outlier_summary[col] = len(outliers)

outlier_summary



{'Number_of_Riders': 0,
 'Number_of_Drivers': 10,
 'Number_of_Past_Rides': 0,
 'Average_Ratings': 0,
 'Expected_Ride_Duration': 0,
 'Historical_Cost_of_Ride': 0}

In [14]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower,
                       np.where(df[col] > upper, upper, df[col]))
    print(f"Outliers in {col}: {len(df[(df[col] < lower) | (df[col] > upper)])}")


Outliers in Number_of_Riders: 0
Outliers in Number_of_Drivers: 0
Outliers in Number_of_Past_Rides: 0
Outliers in Average_Ratings: 0
Outliers in Expected_Ride_Duration: 0
Outliers in Historical_Cost_of_Ride: 0


In [15]:


df["Price"] = df["Historical_Cost_of_Ride"]
df["Completed_Rides"] = df[["Number_of_Riders", "Number_of_Drivers"]].min(axis=1)
df["Booking_Intents"] = df["Number_of_Riders"]
df["Cancelled_Rides"] = df["Booking_Intents"] - df["Completed_Rides"]
df["Cost"] = df["Price"] * 0.7   # assume 70% of fare is operating cost




df["Revenue"] = df["Price"] * df["Completed_Rides"]
df["Profit"] = (df["Price"] - df["Cost"]) * df["Completed_Rides"]


baseline_price = df["Price"].mean()
df["Baseline_Revenue"] = baseline_price * df["Completed_Rides"]
df["Revenue_Lift_%"] = ((df["Revenue"] - df["Baseline_Revenue"]) / df["Baseline_Revenue"]) * 100


df["Gross_Margin_%"] = ((df["Revenue"] - (df["Cost"] * df["Completed_Rides"])) / df["Revenue"]) * 100


df["Conversion_Rate_%"] = (df["Completed_Rides"] / df["Booking_Intents"]) * 100
df["Cancellation_Rate_%"] = (df["Cancelled_Rides"] / df["Booking_Intents"]) * 100


df["Price_Shift"] = df["Price"].shift(1)
df["Price_Change_%"] = ((df["Price"] - df["Price_Shift"]) / df["Price_Shift"]) * 100
df["Price_Change_Flag"] = np.where(df["Price_Change_%"].abs() > 15, 1, 0)


kpi_summary = {
    "Revenue (₹)": df["Revenue"].sum(),
    "Profit (₹)": df["Profit"].sum(),
    "Revenue Lift (%)": df["Revenue_Lift_%"].mean(),
    "Gross Margin (%)": df["Gross_Margin_%"].mean(),
    "Conversion Rate (%)": df["Conversion_Rate_%"].mean(),
    "Price Change Rate (%)": df["Price_Change_Flag"].mean() * 100,
    "Cancellation Rate (%)": df["Cancellation_Rate_%"].mean()
}

kpi_table = pd.DataFrame([kpi_summary])


print("KPI Summary Table:")
print(kpi_table.to_string(index=False))



KPI Summary Table:
 Revenue (₹)   Profit (₹)  Revenue Lift (%)  Gross Margin (%)  Conversion Rate (%)  Price Change Rate (%)  Cancellation Rate (%)
1.013314e+07 3.039942e+06     -4.206413e-15              30.0            43.763674                   86.6              56.236326
